In [2]:
import pandas as pd
import numpy as np
import json
import os

from scripts.train_is_model import *
from scripts.is_model_inference import *

# Sample of images for annotation

In [3]:
intersections = pd.read_csv('data/processed/collisions_merged.csv', low_memory=False)
intersections

,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_ID,PERSON_TYPE,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,...,street1Bearing,street2Bearing,adminName2,postalcode,countryCode,street1,street2,adminName1,distance_site_int,boro_name
0,10249006,4229554,2019-10-26,09:43:00,31aa2bc0-f545-444f-8cdb-f1cb5cf00b89,Occupant,Unspecified,19141108.0,NaN,NaN,...,103,201,Queens,11411.0,US,Linden Blvd,233rd St,New York,0.000161,Queens
1,10255054,4230587,2019-10-25,15:15:00,4629e500-a73e-48dc-b8fb-53124d124b80,Occupant,Unspecified,19144075.0,33.0,Not Ejected,...,164,71,Queens,11385.0,US,Fresh Pond Rd,Catalpa Ave,New York,0.000095,Queens
2,6650180,3565527,2016-11-21,13:05:00,2782525,Occupant,Unspecified,NaN,NaN,NaN,...,265,175,Richmond,10314.0,US,Westwood Ave,Martin Ave,New York,0.000277,Staten Island
3,10253606,4230743,2019-10-24,19:15:00,84bcb3a7-d201-4c61-9e30-fe29268c1074,Occupant,Injured,19143343.0,27.0,Not Ejected,...,36,126,Kings,11237.0,US,Jefferson Ave,Irving Ave,New York,0.000098,Brooklyn
4,10251336,4230047,2019-10-26,16:45:00,21064a07-a945-49d0-af97-5446801b20ce,Occupant,Unspecified,19142198.0,41.0,NaN,...,85,175,Kings,11210.0,US,Foster Ave,Nostrand Ave,New York,0.000150,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306198,13075253,4700981,2024-02-07,19:01:00,02812d77-0651-4ec8-b16f-92ef8e973440,Occupant,Unspecified,20705667.0,55.0,Not Ejected,...,53,11,New York,NaN,US,E 23rd St,Ave C,New York,0.000437,Manhattan
3306199,13075821,4749919,2024-08-20,20:00:00,58c85c87-a0d3-4159-ae7b-224bbb4f2a4f,Occupant,Unspecified,20706007.0,45.0,Not Ejected,...,221,128,Kings,11220.0,US,3rd Ave,49th St,New York,0.000478,Brooklyn
3306200,13075602,4738538,2024-07-04,20:16:00,b57fa6a3-9577-4e59-ac93-fe407bbdce58,Pedestrian,Injured,NaN,14.0,NaN,...,138,242,Queens,NaN,US,Woodhaven Blvd,Forest Park Dr,New York,0.000114,Queens
3306201,13075819,4749919,2024-08-20,20:00:00,06060ebf-1117-4a31-a3e8-efae535d1493,Occupant,Unspecified,20706007.0,36.0,Not Ejected,...,221,128,Kings,11220.0,US,3rd Ave,49th St,New York,0.000478,Brooklyn


In [4]:
intersections.columns

Index(['UNIQUE_ID', 'COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'PERSON_ID',
       'PERSON_TYPE', 'PERSON_INJURY', 'VEHICLE_ID', 'PERSON_AGE', 'EJECTION',
       'EMOTIONAL_STATUS', 'BODILY_INJURY', 'POSITION_IN_VEHICLE',
       'SAFETY_EQUIPMENT', 'PED_LOCATION', 'PED_ACTION', 'COMPLAINT',
       'PED_ROLE', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2',
       'PERSON_SEX', 'YEAR', 'CRASH DATE', 'CRASH TIME', 'ZIP CODE',
       'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME',
       'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'VEHICLE

In [5]:
intersections.boro_name.value_counts()

boro_name
Brooklyn         1059433
Queens            932273
Manhattan         642852
Bronx             509813
Staten Island     161832
Name: count, dtype: int64

In [6]:
intersections.value_counts(subset=['boro_name', 'dupe_key']).groupby('boro_name', group_keys=False).apply(lambda x: x.sample(10))

boro_name      dupe_key            
Bronx          40.814363_-73.896681      67
               40.871329_-73.849927      70
               40.813079_-73.846097       6
               40.877893_-73.850919      55
               40.826492_-73.907739      95
               40.851803_-73.889518      18
               40.859963_-73.906252      25
               40.829344_-73.84829        7
               40.850178_-73.865221      47
               40.817388_-73.922758    1028
Brooklyn       40.592101_-73.956077      16
               40.668974_-73.906673     131
               40.646503_-74.023205      90
               40.694334_-73.976983      11
               40.639225_-74.002091      77
               40.646788_-74.008568     182
               40.636392_-74.030666       8
               40.672618_-74.000595      99
               40.712864_-73.940128       9
               40.622614_-73.986667     114
Manhattan      40.791622_-73.948846     250
               40.755869_-73.998208     

### High accident ints

In [7]:
intersections = pd.read_csv('data/processed/intersection_collision_frequency.csv')
intersections

,lat,lng,intersection,lat_int,lng_int,mtfcc1,mtfcc2,adminCode1,distance,bearing,...,total_collisions_sum,ped_injuries_sum,cyc_injuries_sum,pedcyc_injuries_sum,total_injuries_sum,ped_killed_sum,cyc_killed_sum,pedcyc_killed_sum,total_killed_sum,boro_name
0,40.744167,-73.912480,True,40.744167,-73.912480,S1400,S1400,NY,0.00,0,...,44.0,5.0,1.0,6.0,14.0,0.0,0.0,0.0,0.0,Queens
1,40.681686,-73.871570,True,40.681686,-73.871570,S1400,S1400,NY,0.00,0,...,192.0,19.0,3.0,22.0,130.0,0.0,0.0,0.0,0.0,Brooklyn
2,40.811651,-73.914474,True,40.811651,-73.914474,S1400,S1400,NY,0.00,0,...,38.0,4.0,0.0,4.0,14.0,0.0,0.0,0.0,0.0,Bronx
3,40.692326,-73.744290,True,40.692326,-73.744290,S1400,S1400,NY,0.00,0,...,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,Queens
4,40.666060,-73.958630,True,40.666060,-73.958630,S1400,S1400,NY,0.00,0,...,41.0,6.0,0.0,6.0,20.0,0.0,0.0,0.0,0.0,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39157,40.852615,-73.903510,True,40.852495,-73.903591,S1400,S1400,NY,0.01,207,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bronx
39158,40.590115,-73.990610,True,40.590180,-73.990455,S1400,S1400,NY,0.01,61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brooklyn
39159,40.657257,-73.897910,True,40.657391,-73.897930,S1400,S1400,NY,0.01,353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brooklyn
39160,40.701256,-74.014320,True,40.701248,-74.014143,S1100,S1400,NY,0.01,93,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Manhattan


In [8]:
intersections.columns

Index(['lat', 'lng', 'intersection', 'lat_int', 'lng_int', 'mtfcc1', 'mtfcc2',
       'adminCode1', 'distance', 'bearing', 'placename', 'street1Bearing',
       'street2Bearing', 'adminName2', 'postalcode', 'countryCode', 'street1',
       'street2', 'adminName1', 'distance_site_int', 'dupe_key',
       'ped_collisions_sum', 'cyc_collisions_sum', 'pedcyc_collisions_sum',
       'total_collisions_sum', 'ped_injuries_sum', 'cyc_injuries_sum',
       'pedcyc_injuries_sum', 'total_injuries_sum', 'ped_killed_sum',
       'cyc_killed_sum', 'pedcyc_killed_sum', 'total_killed_sum', 'boro_name'],
      dtype='object')

In [9]:
intersections.sort_values(by='ped_collisions_sum', ascending=False)[['dupe_key', 'boro_name', 'ped_collisions_sum']].head(20)

,dupe_key,boro_name,ped_collisions_sum
11995,40.807773_-73.945482,Manhattan,88.0
15212,40.696986_-73.935222,Brooklyn,70.0
7260,40.737682_-73.852048,Queens,70.0
37647,40.861634_-73.891447,Bronx,67.0
6652,40.668798_-73.931119,Brooklyn,65.0
11205,40.822837_-73.941917,Manhattan,63.0
2821,40.66895_-73.933888,Brooklyn,62.0
17048,40.862749_-73.904934,Bronx,56.0
14232,40.862756_-73.90108,Bronx,56.0
11904,40.821637_-73.939077,Manhattan,55.0


In [10]:
intersections.sort_values(by='cyc_collisions_sum', ascending=False)[['dupe_key', 'boro_name', 'cyc_collisions_sum']].head(20)

,dupe_key,boro_name,cyc_collisions_sum
14750,40.771782_-73.979193,Manhattan,58.0
18539,40.651511_-74.007335,Brooklyn,53.0
2716,40.718145_-73.993828,Manhattan,53.0
19545,40.753244_-73.966616,Manhattan,52.0
21705,40.663517_-73.95721,Brooklyn,50.0
37638,40.719895_-73.992936,Manhattan,50.0
12992,40.759982_-73.964801,Manhattan,49.0
26679,40.813375_-73.956269,Manhattan,44.0
5974,40.820438_-73.936227,Manhattan,43.0
16061,40.791664_-73.964685,Manhattan,43.0


In [11]:
intersections.sort_values(by='total_collisions_sum', ascending=False)[['dupe_key', 'boro_name', 'total_collisions_sum']].head(30)

,dupe_key,boro_name,total_collisions_sum
36942,40.861916_-73.912704,Bronx,1573.0
18205,40.675737_-73.896852,Brooklyn,1005.0
28877,40.668496_-73.925598,Brooklyn,1002.0
36659,40.696127_-73.984514,Brooklyn,894.0
16253,40.678163_-73.897475,Brooklyn,770.0
21638,40.658576_-73.890621,Brooklyn,702.0
24284,40.820185_-73.890677,Bronx,694.0
36988,40.691608_-73.999343,Brooklyn,673.0
14729,40.659651_-73.77383,Queens,604.0
5795,40.826277_-73.859703,Bronx,570.0


In [13]:
intersections.sort_values(by='cyc_collisions_sum', ascending=True)[['dupe_key', 'boro_name', 'cyc_collisions_sum']].head(30)

,dupe_key,boro_name,cyc_collisions_sum
39161,40.83881_-73.913716,Bronx,0.0
31780,40.830877_-73.83434,Bronx,0.0
20007,40.820364_-73.870501,Bronx,0.0
20006,40.842549_-73.820461,Bronx,0.0
20005,40.863614_-73.928206,Manhattan,0.0
20004,40.884969_-73.830337,Bronx,0.0
20003,40.898252_-73.85785,Bronx,0.0
20002,40.849913_-73.929633,Manhattan,0.0
20001,40.865311_-73.903322,Bronx,0.0
20000,40.605652_-73.972398,Brooklyn,0.0


In [30]:
intersections.sort_values(by='cyc_collisions_sum', ascending=False)[['dupe_key', 'boro_name', 'cyc_collisions_sum']].iloc[5000:5020]

,dupe_key,boro_name,cyc_collisions_sum
36499,40.694507_-73.996197,Brooklyn,3.0
14247,40.767914_-73.925417,Queens,3.0
7748,40.793552_-73.945324,Manhattan,3.0
14456,40.614447_-73.980805,Brooklyn,3.0
36519,40.722334_-74.011842,Manhattan,3.0
9373,40.823191_-73.889484,Bronx,3.0
9293,40.706708_-74.002329,Manhattan,3.0
36410,40.72756_-73.991364,Manhattan,3.0
94,40.727789_-73.895499,Queens,3.0
14502,40.734296_-73.850304,Queens,3.0


In [32]:
intersections.sort_values(by='ped_collisions_sum', ascending=False)[['dupe_key', 'boro_name', 'ped_collisions_sum']].iloc[5000:5020]

,dupe_key,boro_name,ped_collisions_sum
5981,40.765358_-73.744282,Queens,5.0
5978,40.770424_-73.962211,Manhattan,5.0
11344,40.605379_-73.9821,Brooklyn,5.0
11341,40.743195_-73.936722,Queens,5.0
19805,40.717931_-73.736958,Queens,5.0
20857,40.596616_-73.97651,Brooklyn,5.0
20845,40.752054_-73.8307,Queens,5.0
31652,40.629655_-73.96508,Brooklyn,5.0
31440,40.755746_-73.76615,Queens,5.0
10898,40.746053_-73.78812,Queens,5.0


In [33]:
intersections.sort_values(by='cyc_collisions_sum', ascending=False)[['dupe_key', 'boro_name', 'cyc_collisions_sum']].iloc[1000:1010]

,dupe_key,boro_name,cyc_collisions_sum
20171,40.69133_-73.951757,Brooklyn,10.0
7352,40.737886_-73.980905,Manhattan,10.0
21970,40.65973_-73.956801,Brooklyn,10.0
12400,40.75223_-73.939102,Queens,10.0
16142,40.683418_-73.981993,Brooklyn,10.0
16459,40.698188_-73.961899,Brooklyn,10.0
12531,40.762914_-73.969801,Manhattan,10.0
16047,40.621672_-73.991256,Brooklyn,10.0
12568,40.636134_-73.972735,Brooklyn,10.0
16003,40.750417_-73.897728,Queens,10.0


In [34]:
intersections.sort_values(by='ped_collisions_sum', ascending=False)[['dupe_key', 'boro_name', 'ped_collisions_sum']].iloc[1000:1010]

,dupe_key,boro_name,ped_collisions_sum
1396,40.626184_-73.994146,Brooklyn,14.0
12000,40.795541_-73.948095,Manhattan,14.0
633,40.754505_-73.965696,Manhattan,14.0
16061,40.791664_-73.964685,Manhattan,14.0
15854,40.723625_-74.004786,Manhattan,14.0
6448,40.766894_-73.921391,Queens,14.0
17624,40.73986_-73.995018,Manhattan,14.0
16112,40.865429_-73.86736,Bronx,14.0
9527,40.765545_-73.954701,Manhattan,14.0
6479,40.687789_-73.820165,Queens,14.0


# Identify annotated images with certain classes

In [14]:
import os
import json

# Load the JSON file
annotations_dir = 'data/annotations/'
annotations_files = os.listdir(annotations_dir)

target_images = []

for annotations in annotations_files:
    # Load the JSON data
    file_path = os.path.join(annotations_dir, annotations)
    with open(file_path, 'r') as f:
        data = json.load(f)

    target_classes = ['overpass']
    if 'shapes' in data:
        target_shapes = [shape for shape in data['shapes'] if shape['label'] in target_classes]

    if target_shapes:
        target_images.append(annotations.replace('.json', ''))

target_images

['40.605652_-73.972398',
 '40.630821_-73.886357',
 '40.642243_-74.020664',
 '40.681217_-73.882107',
 '40.691554_-73.999136',
 '40.691608_-73.999343',
 '40.696986_-73.935222',
 '40.700031_-73.961875',
 '40.700725_-73.941931',
 '40.701473_-73.989647',
 '40.737682_-73.852048',
 '40.748914_-73.937395',
 '40.792928_-73.947896',
 '40.803242_-73.930745',
 '40.804545_-73.912091',
 '40.81308_-73.898215',
 '40.815733_-73.895241',
 '40.820185_-73.890677',
 '40.826277_-73.859703',
 '40.861916_-73.912704',
 '40.862756_-73.90108',
 '40.877513_-73.866874',
 '40.880783_-73.883464',
 '40.884186_-73.833618',
 '40.893394_-73.857368']

# Remove Lane Labels for Good

In [ ]:
# import os
# import json

# # Load the JSON file
# annotations_dir = 'data/annotations/'
# annotations_files = os.listdir(annotations_dir)

# for annotations in annotations_files:
#     # Load the JSON data
#     file_path = os.path.join(annotations_dir, annotations)
#     with open(file_path, 'r') as f:
#         data = json.load(f)

#     # Keep annotations that are NOT in the specified classes
#     excluded_classes = {"intersection", "road"}
#     filtered_shapes = [shape for shape in data['shapes'] if shape['label'] not in excluded_classes]

#     # Update the JSON data
#     data['shapes'] = filtered_shapes

#     # Save the cleaned JSON
#     with open(file_path, 'w') as f:
#         json.dump(data, f, indent=4)

KeyError: 'shapes'

In [ ]:
# %run scripts/get_google_images.py

Processing 16163 records out of 39162 total intersections...
Processed 0 images...Elapsed time: 0.01 minutes
Processed 500 images...Elapsed time: 6.74 minutes
Processed 1000 images...Elapsed time: 13.51 minutes
Processed 1500 images...Elapsed time: 20.24 minutes
Processed 2000 images...Elapsed time: 26.89 minutes
Processed 2500 images...Elapsed time: 33.49 minutes
Processed 3000 images...Elapsed time: 40.07 minutes
Processed 3500 images...Elapsed time: 46.46 minutes
Processed 4000 images...Elapsed time: 53.10 minutes
Processed 4500 images...Elapsed time: 59.57 minutes
Processed 5000 images...Elapsed time: 66.04 minutes
Processed 5500 images...Elapsed time: 76.89 minutes
Processed 6000 images...Elapsed time: 83.35 minutes
Processed 6500 images...Elapsed time: 94.73 minutes
Processed 7000 images...Elapsed time: 101.15 minutes
Processed 7500 images...Elapsed time: 111.62 minutes
Processed 8000 images...Elapsed time: 118.03 minutes
Processed 8500 images...Elapsed time: 124.46 minutes
Proce

# Change dimensions in annotations

In [ ]:
# import json
# import os

# def update_image_dimensions(annotation_folder, new_width, new_height):
#     annotation_files = [f for f in os.listdir(annotation_folder) if '_' in f]
#     for filename in annotation_files:
#         if filename.endswith(".json"):
#             filepath = os.path.join(annotation_folder, filename)
#             with open(filepath, 'r') as f:
#                 data = json.load(f)
            
#             # Update the dimensions
#             data['imageWidth'] = new_width
#             data['imageHeight'] = new_height
            
#             # Save back to the file
#             with open(filepath, 'w') as f:
#                 json.dump(data, f, indent=4)
            
#             print(f"Updated {filename}")

# # Update the annotations
# annotation_folder = "data/annotations"  # Replace with your actual path
# new_width = 1280
# new_height = 1280

# # update_image_dimensions(annotation_folder, new_width, new_height)

Updated 40.542_-74.187925.json
Updated 40.556_-74.194743.json
Updated 40.557637_-74.13811.json
Updated 40.571337_-74.128729.json
Updated 40.594527_-73.934006.json
Updated 40.609809_-73.922498.json
Updated 40.61622_-74.083234.json
Updated 40.618772_-74.123391.json
Updated 40.624_-74.036726.json
Updated 40.627138_-74.07638.json
Updated 40.628838_-74.126173.json
Updated 40.630592_-73.975504.json
Updated 40.63171_-74.116666.json
Updated 40.636735_-74.134282.json
Updated 40.642243_-74.020664.json
Updated 40.666_-73.816474.json
Updated 40.668798_-73.931119.json
Updated 40.66895_-73.933888.json
Updated 40.669_-73.803565.json
Updated 40.670724_-73.87518.json
Updated 40.674_-73.982277.json
Updated 40.677_-73.819641.json
Updated 40.681217_-73.882107.json
Updated 40.682_-73.968664.json
Updated 40.683249_-73.818676.json
Updated 40.68355_-73.98438.json
Updated 40.688_-73.997562.json
Updated 40.696935_-73.803572.json
Updated 40.696986_-73.935222.json
Updated 40.69746_-73.99302.json
Updated 40.700725

In [ ]:
# import json
# import os

# def scale_annotations(file_path, scale_factor):
#     # Load the JSON file
#     with open(file_path, 'r') as f:
#         data = json.load(f)
    
#     # # Update image dimensions
#     # data['imageWidth'] *= scale_factor
#     # data['imageHeight'] *= scale_factor

#     # Scale each point in the shapes
#     for shape in data['shapes']:
#         scaled_points = [
#             [point[0] * scale_factor, point[1] * scale_factor] 
#             for point in shape['points']
#         ]
#         shape['points'] = scaled_points
    
#     # Save the updated file
#     with open(file_path, 'w') as f:
#         json.dump(data, f, indent=4)
#     print(f"Updated file saved to: {file_path}")

# # Usage
# file_path = "data/annotations/40.7_-73.957226.json"  # Replace with your file path
# scale_factor = 2  # Adjust to match your scale change

# # scale_annotations(file_path, scale_factor)

Updated file saved to: data/annotations/40.7_-73.957226.json


# Refresh imageData in Labelme

In [ ]:
# import os
# import json
# import base64
# from PIL import Image

# # Paths
# annotation_folder = "data/annotations"  # Replace with your annotation folder
# image_folder = "data/images"  # Replace with your updated image folder

# annotation_files = [f for f in os.listdir(annotation_folder) if '_' in f]
# for annotation_file in annotation_files:
#     if annotation_file.endswith(".json"):
#         annotation_path = os.path.join(annotation_folder, annotation_file)
#         with open(annotation_path, 'r') as f:
#             data = json.load(f)
        
#         # Get the corresponding image file
#         image_path = data['imagePath']
#         if os.path.exists(image_path):
#             # Encode the image in base64
#             with open(image_path, "rb") as img_f:
#                 encoded_data = base64.b64encode(img_f.read()).decode('utf-8')
            
#             # Update the imageData
#             data['imageData'] = encoded_data
            
#             # Save updated annotation
#             with open(annotation_path, 'w') as f:
#                 json.dump(data, f, indent=4)
            
#             print(f"Updated {annotation_file}")
#         else:
#             print(f"Image {data['imagePath']} not found for annotation {annotation_file}")

Updated 40.542_-74.187925.json
Updated 40.556_-74.194743.json
Updated 40.557637_-74.13811.json
Updated 40.571337_-74.128729.json
Updated 40.594527_-73.934006.json
Updated 40.609809_-73.922498.json
Updated 40.61622_-74.083234.json
Updated 40.618772_-74.123391.json
Updated 40.624_-74.036726.json
Updated 40.627138_-74.07638.json
Updated 40.628838_-74.126173.json
Updated 40.630592_-73.975504.json
Image ..\images\40.630821_-73.886357.png not found for annotation 40.630821_-73.886357.json
Updated 40.63171_-74.116666.json
Updated 40.636735_-74.134282.json
Updated 40.642243_-74.020664.json
Image ..\images\40.651511_-74.007335.png not found for annotation 40.651511_-74.007335.json
Image ..\images\40.652633_-74.006174.png not found for annotation 40.652633_-74.006174.json
Image ..\images\40.65616_-73.767353.png not found for annotation 40.65616_-73.767353.json
Image ..\images\40.658576_-73.890621.png not found for annotation 40.658576_-73.890621.json
Image ..\images\40.659651_-73.77383.png not f

In [ ]:
# annotation_files = [f for f in os.listdir(annotation_folder) if '_' in f and '40.7_-73.957226' not in f]

# for filename in annotation_files:
#     file_path = os.path.join(annotation_folder, filename)
#     scale_annotations(file_path, scale_factor)

Updated file saved to: data/annotations\40.542_-74.187925.json
Updated file saved to: data/annotations\40.556_-74.194743.json
Updated file saved to: data/annotations\40.557637_-74.13811.json
Updated file saved to: data/annotations\40.571337_-74.128729.json
Updated file saved to: data/annotations\40.594527_-73.934006.json
Updated file saved to: data/annotations\40.609809_-73.922498.json
Updated file saved to: data/annotations\40.61622_-74.083234.json
Updated file saved to: data/annotations\40.618772_-74.123391.json
Updated file saved to: data/annotations\40.624_-74.036726.json
Updated file saved to: data/annotations\40.627138_-74.07638.json
Updated file saved to: data/annotations\40.628838_-74.126173.json
Updated file saved to: data/annotations\40.630592_-73.975504.json
Updated file saved to: data/annotations\40.63171_-74.116666.json
Updated file saved to: data/annotations\40.636735_-74.134282.json
Updated file saved to: data/annotations\40.642243_-74.020664.json
Updated file saved to: 